# Tutorial 1: MDITRE workflow on 16s-based data
In this tutorial we demonstrate how to use MDITRE on 16S rRNA amplicon sequencing data, using the dataset from David et al., (2014). The study from David et al., (2014) tracked the gut microbiome composition of 20 healthy adults before,
during, and after a 5-day period of consuming exclusively plant-based or exclusively animal-based diets.

## Loading and preprocessing the data
The following files are required for the data loading step. For other optional files that are accepted by the software for data loading and optional preprocessing features, please refer to our user manual [here](https://github.com/gerberlab/mditre/blob/master/mditre/docs/config_doc.pdf).

### Required files
#### Abundance data
A CSV file containing the microbial abundances, formatted with the first row providing OTU IDs and the first column providing sample IDs. See [here](./datasets/raw/david/abundance.csv) for the abundance data file for this dataset.

#### Sample metadata
A CSV file that specifies an associated subject ID and timepoint for each sample ID. See [here](./datasets/raw/david/sample_metadata.csv) for the sample metadata file for this dataset.

#### Subject metadata
A CSV file that gives information about each subject, (including the value of whatever variable will be used as the host outcome for prediction (e.g., Plant-diet or Animal-diet in the David et al). See [here](./datasets/raw/david/subject_data.csv) for the subject metadata file for this dataset.

#### Pplacer output
A ```.jplace``` file created by placing the representative sequences from each OTU on a reference tree of known 16S rRNA amplicon sequences using the [pplacer](https://matsen.fhcrc.org/pplacer/) package. See [here](./datasets/raw/david/placements.jplace) for the pplacer output file for this dataset. This will be used to create a tree of phylogenetic placements of the OTUs which is required by the model.

### Preparing a config file
The software requires user-defined config file with the desired data loading and preprocessing options. A pre-defined config file for this dataset is found [here](./datasets/raw/david/david_benchmark.cfg). Looking into this file, one can see a section ```[data]``` that contains the flags specifying the locations of the required data files and also a ```[preprocessing]``` section with the optional data preprocessing flags, described in detail in our user manual. The following are the flags the user will always need to specify in the config file.

- <strong>tag</strong>: A short string used to generate the filename of the processed dataset (written to the current directory) in the following format ```<tag>_dataset_object.pickle```
- <strong>abundance_data</strong>: Location of the abundance data file
- <strong>sample_metadata</strong>: Location of sample metadata file
- <strong>subject_data</strong>: Location of subject metadata file
- <strong>jplace_file</strong>: Location of the pplacer generated .jplace file
- <strong>outcome_variable</strong>: A string specifying which of the columns in the subject metadata table encodes the host status variable for prediction (For example, 'diet').
- <strong>outcome_positive_value</strong>: The value of the host status variable that corresponds to a 'positive' outcome (for example, 'Plant'; the choice doesn't affect the model and is just used for labeling outputs)

### Running the data loading/preprocessing utility
After defining the config file, we run the 'preprocess' utility, which produces a serialized python pickle object containing the pre-processed data, in this case ```david_diet_dataset_object.pickle``` in the current folder.

In [ ]:
# Load the required packages
# Make sure to have mditre package installed via pip prior to this step!
from mditre.data_utils import preprocess, ConfigParser, pickle

In [ ]:
# Define the location of the config file
filename_16s_data_cfg = './datasets/raw/david/david_benchmark.cfg'

In [ ]:
# Load the config file using ConfigParser package
config_16s = ConfigParser.ConfigParser()
config_16s.read(filename_16s_data_cfg)

In [ ]:
# Load and preprocess 16s data
dataset_16s = preprocess(config_16s)

## Running the model on preprocessed data
We are now ready to run the MDITRE model for analysis, which takes the pickle file with the preprocessing data from the previous step. Running the MDITRE model will output the following information.
- The predictive ability of the model using F1 score as the quantitative measure
- A serialized pickle object containing all the information needed to interpret and visualize the learned rules.

### Quick start with default options
We recommend running MDITRE using cross-validation to assess the generalization capability of the model. Since the pytorch multiprocessing module does not support running from a Jupyter notebook, we included a python script [here ](./model_run_tutorial.py) to run the model on the prepared dataset from the previous step using default configuration options on the command line. We provide the following command that executes the script and runs a 5-fold cross-validation procedure using multiprocessing (1 process per fold) and returns logs pertaining to some training diagnostics (useful for debugging) and finally the F1 score.

```python -m torch.distributed.launch --nproc_per_node=5 model_run_tutorial.py --data ./datasets/david_agg_filtered.pickle --data_name david_test --is_16s```

The MDITRE software is optimized to run on a GPU. However, it can also run on a CPU. On a CPU, it should still take only a few minutes to run this example in multiprocessing mode. If the user has limited CPU cores, they can set the "nproc_per_node" to a lower number. The full description of the command line arguments accepted by the model code is located in the documentation [here](https://github.com/gerberlab/mditre/blob/master/mditre/docs/config_doc.pdf). A brief summary of the required options to be passed as arguments to the command are as follows.

- <strong>nproc_per_node</strong>: Number of parallel processes to be used for cross-validation procedure
- <strong>data</strong>: Location of the preprocessed data pickle file from the previous step
- <strong>data_name</strong>: A string used to create output file directory
- <strong>is_16s</strong>: Required if using 16S rRNA amplicon based dataset (omit if using metagenomics based data)

After executing the above command successfully, you can find the final F1 score from the cross-validation (0.82) and also the saved pickle file containing the learned rule information here ```david_rules_dump.pickle```, which will be used in the following rule visualization tutorial.

## Rule Visualization via GUI Tutorial
Now we demonstrate the rule visualization GUI (Graphical User Interface) capabilities of the MDITRE software. After successfully running the model code on a given dataset, the user can interactively explore the learned rules via the GUI. We show an example walkthrough of the GUI on the learned rules obtained from running the MDITRE model in the previous step.

In [ ]:
# Import packages
from mditre.rule_viz import *
# Make the notebook pop up a separate matplotlib window (instead of displaying inline)
%matplotlib qt

### Specify the location of rule output pickle file output by the model code
After a successful run of the model code, the software saves a pickle file with suffix "rules_dump.pickle" in a specified location. This file contains all the necessary data to visualize the learned rules via the GUI.

In [ ]:
rules_path = './david_rules_dump.pickle'

### Load the pickle file
The loaded file is a dictionary that will be input into the GUI code.

In [ ]:
with open(rules_path, 'rb') as f:
    rule_dict = pickle.load(f)

### Invoke the RuleVisualizer utility
This opens a separate window with the rule visualization. We describe the workflow in the following steps.

In [ ]:
rv = RuleVisualizer(rule_dict, './')
rv.plot_log_odds()

## Rule Visualizer workflow
- After running the rule visualizer utlity as shown before, a separate window pops up showing the log-odds of all the learned rules as heatmaps. In this example, we see that the model was able to learn 2 rules (rule 1 and rule 2). The topmost panel shows the log-odds of individual rules. This is followed by 2 panels showing the predicted log-odds of each rule individually for each subject. The next panel shows the combined log-odds of both rules for each subjects. Finally, a panel showing ground truth labels for each subject (Plant vs Animal diet). A rule with positive log-odds is predictive of the positive outcome (in this case Plant-diet) and vice-versa. As we can see, both rules individually and collectively can perfectly classify the subjects based on their outcomes. Next, in order to view the learned rules, one can click on the button (that says "Click").
- Clicking on a button to the left of the heatmap of rule 1 opens a separate window. This window shows the text of the rules on the top, followed by a panel showing the detector activations for each subject as a heatmap. As we see, the rule is predictive of Animal-diet and therefore the detector is active (True) for all the subjects with Animal-diet. Since this rule contains only 1 detector, we omit showing the conjunction of all the detectors in a separate panel.
- Next, the user can click on the button to the left of the detector heatmap, which opens a separate window showing the average rate of change of abundances (slope) over time for the 2 groups of subjects of the selected taxa (shown as a tree on the right), along with the heatmaps of the slope within the selected time window (day 10 - 15). <strong>Note, if the figure does not fit the screen, please click on "Configure subplots" button (located to the left of the zoom tool on the top left), and then click "tight layout" option.</strong> The purpose of this visualization is to allow the user to explore the differences in the temporal dynamics of the selected taxa both within and outside the selected time window between the subjects. One can see that the abundances of subjects with Animal diet increase at a higher rate (colored towards red) than the ones with Plant-diet (colored towrads blue) during days 10 - 15. An opposite effect can be observed during days 6 - 9, which is the period of the actual dietary intervention. This allows the user to gain insights into the effect of the dietary intervention on the microbiome.
- We also provide a button located in the bottom right, which when clicked opens a separate window with the full phylogenetic tree of taxa. Using the zoom tool located at the top left of the new window, one can see the selected taxa (with red branches) in the context of the other nearby taxa in the full phylogenetic tree.

In [ ]:
# Close the GUI
plt.close("all")